In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.stats import gaussian_kde
from scipy.signal import argrelextrema

In [ ]:
# final kmatrix
with open('../v2/final_kernel_matrix/complete_kernel_matrix_v2.pkl', 'rb') as f:
    kmatrix = pickle.load(f)

In [ ]:
similarity_matrix = np.array(kmatrix)
similarity_matrix = (similarity_matrix + similarity_matrix.T) / 2

np.fill_diagonal(similarity_matrix, 0) # Remove self-similarities

# Flatten the matrix and remove zero values (self-similarities already removed)
similarity_scores = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]

plt.figure(figsize=(10, 6))
plt.hist(similarity_scores, bins=50, alpha=0.75, color='blue', range=(0, 0.01))
plt.title('Histogram of Similarity Scores')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# Step 1: KDE to estimate the probability density function
kde = gaussian_kde(similarity_scores)
x_grid = np.linspace(0, 0.01, 1000)  # Define a grid over our range of interest
kde_pdf = kde.evaluate(x_grid)

# Step 2: Calculate the second derivative
kde_pdf_second_derivative = np.gradient(np.gradient(kde_pdf))

# Step 3: Identify inflection points (changes in the sign of the second derivative)
# We're interested in local minima and maxima as potential inflection points
inflection_points_indices = argrelextrema(kde_pdf_second_derivative, np.greater)[0]  # Local maxima
inflection_points_indices = np.append(inflection_points_indices, argrelextrema(kde_pdf_second_derivative, np.less)[0])  # Local minima
inflection_points = x_grid[inflection_points_indices]

# Plotting for visualization
plt.figure(figsize=(12, 7))

# KDE plot
plt.plot(x_grid, kde_pdf, label='KDE', color='blue')

# Second derivative plot
plt.plot(x_grid, kde_pdf_second_derivative, label='Second Derivative', color='green')

# Mark inflection points
for inflection_point in inflection_points:
    plt.axvline(x=inflection_point, color='red', linestyle='--')

plt.title('KDE and Second Derivative of Similarity Scores')
plt.xlabel('Similarity Score')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

inflection_points

In [ ]:
five = np.percentile(similarity_matrix, 5)
ninety_five = np.percentile(similarity_matrix, 95)

inflection_points = np.append(inflection_points, [ninety_five, five])

In [ ]:
with open('inflection_points.pkl', 'wb') as f:
    pickle.dump(inflection_points, f)